In [1]:
%matplotlib inline

import pandas as pd
import pickle
import time
import importlib
import numpy as np
import importlib


import sys
sys.path.append('/home/ngr/gdrive/wearables/scripts')
import data as weardata
import utils as wearutils

In [2]:
timer = wearutils.timer()
timer.start()
data = weardata.load_datadict()
print('Dict data loaded in {}-s'.format(timer.stop()))

Dict data loaded in 377.23827743530273-s


In [101]:
sns.ecdfplot(x='n_days', data=nt)

AttributeError: module 'seaborn' has no attribute 'ecdfplot'

In [20]:
# check differences

def chk_tdeltas(data, truncate_at_day=None, leading_pad=True, drop_first_day=True, verbose=True):
    '''Check for the temporal differences between consecutive measurements.
    
    Arguments:
      truncate_at_day (int): (optional, Default=None) in units of d, truncate after x days. None keeps all data 
      leading_pad (bool): (optional, Default=True) add pads to align ts so first time is 12 AM
      drop_first_day (bool): (optional, Default=True) drop the first day since, worst case is measurement
        starting at 12 AM
    '''

    k_irreg_tdelta = []
    irreg_tdelta = []
    keys2keep = []
    
    if verbose:
        timer.start()
    for i, (k, v) in enumerate(data.items()):
        ts, x = v[0], v[1]
        if leading_pad:
            zeros2pad = np.zeros((ts[0].hour*60 + ts[0].minute + 1))
            ts = pd.to_datetime(np.linspace(ts[0].replace(hour=0, minute=0).value, ts[0].value, ts[0].hour*60 + ts[0].minute + 1)).to_list() + ts
            x = np.concatenate((zeros2pad, [float(i) for i in x]))
        if drop_first_day:
            # drop 24h/d * 60min/h timepts if 1d drop
            
        unique_tdeltas = np.unique(np.diff(pd.Series(ts)))
        if np.timedelta64(1, 'm') in unique_tdeltas and unique_tdeltas.shape[0] == 1: 
            keys2keep.append(k)
        else:
            k_irreg_tdelta.append(k)
            irreg_tdelta.append(unique_tdeltas)
        if verbose:
            if i % 500 == 0 and i != 0 :
                print('  through m={}\t{:.0f}-s elapsed'.format(i+1, timer.stop()))
    if verbose:
        print('\n{} measurements (out of {}) have multiple t delta'.format(len(k_irreg_tdelta), len(data.keys())))
        npt, nobs = keys2_npt_nobs(keys2keep)
        print('  i.e., {}-pts across {}-measurements have 1min t deltas, whereas'.format(npt, nobs))
        npt, nobs = keys2_npt_nobs(k_irreg_tdelta)
        print('        {}-pts across {}-measurements have more than 1 t delta'.format(npt, nobs))

    if verbose:
        print('  \nunique time deltas:')
        print(pd.value_counts([j for i in irreg_tdelta for j in i]))
    
    return keys2keep, k_irreg_tdelta

In [27]:

def get_ntimepoints(data, leading_pad=True, drop_first_day=True):
    '''Get the number of timepoints (not necessarily minutes)
    
    Arguments:
      data (dict): keys have label, values have [ts, measurements]
      leading_pad (bool): (optional, Default=True) add pads to align ts so first time is 12 AM
      drop_first_day (bool): (optional, Default=True) drop the first day since, worst case is measurement
        starting at 12 AM
        
    NOTE: 
      - we want to get an accurate count of how many days of measurements and cohort sizes can be using, e.g.,
        3d vs. 1wk of data and to see what numbers we can get
    '''
    n_t = pd.DataFrame()
    for k in data.keys():
        pid, ga = k.split('-')
        n_t = n_t.append(pd.DataFrame({'key':k, 'pid':pid, 'GA':ga, 'n_t':len(data[k][0]), }, index=[0]), ignore_index=True)
    n_t['n_days'] = n_t['n_t']/(24*60)
    return n_t